In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.spatial import distance

# Kalman Filter

### Introduction and mathematical formulation

The Kalman filter is an algorithm that provides estimates of some unknown variables given the measurements observed over time containing statistical noise or other inaccuracies. It is composed by a set of mathematical equations that implement a predictor-corrector type estimator that is optimal in the sense that it minimizes the estimated error covariance.

The Kalman filter addresses the general problem of trying to estimate the state $x$ ∈ ℜ$^n$ of a discrete-time controlled process that is governed by the linear stochastic
difference equation:

$x_k = Ax_{k-1} +Bu_k +w_{k-1}\quad\mathrm{,} \quad\quad  p(w) ≈ N(0,Q)$        

with a measurement $y$ ∈ ℜ$^m$ that is:

$y_k = Hx_{k} + v_{k}\quad\mathrm{,}  \quad\quad\quad\quad\quad\quad p(v) ≈ N(0,R)$

The random variables $w_k$ and $v_k$  represent the process and measurement noise respectively. They are assumed to be independent of each other, white, and with normal probability distributions.

In practice, the process noise covariance $Q$ and measurement noise $R$ covariance matrices might change with each time step or measurement, however here we assume they are constant. The $nxn$ matrix $A$ relates the state at the previous time step to the state at the current step, in the absence of either a driving function or process noise. The $n×1$ matrix $B$ relates the optional control input $u$ ∈ ℜ$^l$ to the state $x$. The $mxn$ matrix $H$ in the measurement equation relates the state to the
measurement $y_k$.

The Kalman filter process has two steps: the prediction step, where the next state of the system is predicted given the previous measurements, and the update step, where the current state of the system is estimated given the measurement at that time step.
The steps translate to equations as follows:

#### I. Prediction

$X{_k}^- = A_{k-1}X_{k-1} +B_kU_k $

$P{_k}^- = A_{k-1}P_{k-1}A_{k-1}^T +Q_{k-1} $

#### II. Update

$V{_k} = Y_{k} - H_kX{_k}^- $

$S{_k} = H_{k}P_{k-1}H_{k}^T + R_k $

$K{_k} = P{_k}H_{k-1}^TS{_k}^- $ 

$X{_k} = X{_k}^- + K{_k}V_{k} $

$P{_k} = P{_k}^- - K_kS_kK_{k}^T $

where:

• $X{_k}^-$ and $P{_k}^-$ are the predicted mean and covariance of the state, respectively, on the time step k before seeing the measurement.

• $X_k$ and $P_k$ are the estimated mean and covariance of the state, respectively, on time step k after seeing the measurement.

• $Y_k$ is mean of the measurement on time step k.

• $V_k$ is the innovation or the measurement residual on time step k .

• $S_k$ is the measurement prediction covariance on the time step k .

• $K_k$ is the filter gain, which tells how much the predictions should be corrected on time step k. 


### Python code of Kalman Filter 

#### I. Prediction step

This step has to predict the mean $X$ and the covariance $P$ of the system state at the time step $k$ . The Python function 
$\texttt{kf_predict}$ performs the prediction of these output ( $X$ and $P$ ) when giving six input:

$X$ : The mean state estimate of the previous step $(k −1)$. 

$P$ : The state covariance of previous step $(k −1)$.

$A$ : The transition $nxn$ matrix.

$Q$ : The process noise covariance matrix.

$B$ : The input effect matrix.

$U$ : The control input. 

The Python code of this step is given by: 

In [2]:
def kf_predict(X, P, A, Q, B, U):
    X = np.dot(A, X) + np.dot(B, U)
    P = np.dot(A, np.dot(P, A.T)) + Q
    return(X,P)

#### II. Update step

At the time step $k$ , this update step computes the posterior mean $X$ and covariance $P$ of the system state given a new measurement $Y$ . The Python function $\texttt{kf_update}$ performs the update of $X$ and $P$ giving the predicted $X$ and $P$ matrices, the measurement vector $Y$ , the measurement matrix $H$ and the measurement covariance matrix $R$ . The additional input will be:

$K$ : the Kalman Gain matrix

$IM$ : the Mean of predictive distribution of $Y$

$IS$ : the Covariance or predictive mean of $Y$

$LH$ : the Predictive probability (likelihood) of measurement which is computed using the Python function $\texttt{gauss_pdf}$. 

The Python code of these two functions is given by:

In [3]:
def kf_update(X, P, Y, H, R):
    IM = np.dot(H, X)
    IS = R + np.dot(H, np.dot(P, H.T))
    K = np.dot(P, np.dot(H.T, np.linalg.inv(IS)))
    X = X + np.dot(K, (Y-IM))
    P = P - np.dot(K, np.dot(IS, K.T))
    LH = gauss_pdf(Y, IM, IS)
    return (X,P,K,IM,IS,LH)

def gauss_pdf(X, M, S):
    if M.shape[1] == 1:
        DX = X - np.tile(M, X.shape[1])
        E = 0.5 * np.sum(DX * (np.dot(np.linalg.inv(S), DX)), axis=0)
        E = E + 0.5 * M.shape[0] * math.log(2 * math.pi) + 0.5 * math.log(np.linalg.det(S))
        P = math.exp(-E)
    elif X.shape[1] == 1:
        DX = np.tile(X, M.shape[1])- M
        E = 0.5 * np.sum(DX * (np.dot(np.linalg.inv(S), DX)), axis=0)
        E = E + 0.5 * M.shape[0] * math.log(2 * math.pi) + 0.5 * math.log(np.linalg.det(S))
        P = math.exp(-E)
    else:
        DX = X-M
        E = 0.5 * np.dot(DX.T, np.dot(np.linalg.inv(S), DX))
        E = E + 0.5 * M.shape[0] * math.log(2 * math.pi) + 0.5 * math.log(np.linalg.det(S))
        P = math.exp(-E)
        return (P[0],E[0]) 

### Inizialization of the matrices

#### Choose the State Variables X

The first step is to choose our state variables. We are tracking in two dimensions, so we know that we have the two observed variables $x$ and $y$. If we created our Kalman filter using only those two variables the performance would not be very good because we would be ignoring the information velocity can provide to us. We will incorporate velocity into our equations as well. We will represent this as:

$$\mathbf X = \begin{bmatrix}x&y & v_x & v_y\end{bmatrix}$$

where $v_x$ and $v_y$ are respectively the component of velocity $v$ along the x and y-axis.

In order to derive from the know vector $v$ the velocity along the axis x and y, we calculate, for each time step, the angle between the vector $v$ and its components as:

$$\alpha = arctan \frac{y_{c}-y_{c-1}}{x_{c}-x_{c-1}} $$

where:

$(x_c,y_{c})$ are the coordinates of the cell associated with the x-y veichle location

$(x_{c-1},y_{c-1})$ are the coordinates of the cell visited before the current one

Then, we can calculate the two components as:

$v_x = v * cos(\alpha)$

$v_y = v * sin(\alpha)$

In [4]:
#The method velocity return the components of velocity v along the x and y-axis given C, C-1, v and BS_loc.

def acc_BS(i,BS_loc):
    return BS_loc.iloc[int(i)-1]

def velocity(c_id1,cmin1_id1,v,BS_loc):    
    BS_c = np.array([])
    BS_c_min1 = np.array([])
    for i in c_id1:
        BS_c = np.append(BS_c,acc_BS(i,BS_loc))
    BS_c = BS_c.reshape(-1,2)

    for i in cmin1_id1:
        if ( int(i)<= 0):
            BS_c_min1 = np.append(BS_c_min1, acc_BS(c_id1[0],BS_loc))
            continue
        BS_c_min1 = np.append(BS_c_min1,acc_BS(i,BS_loc))
    BS_c_min1 = BS_c_min1.reshape(-1,2)


    num_rows, num_cols = BS_c.shape
    alpha = np.array([])
    for i in range(0,num_rows):
        if ( BS_c[i,1] == BS_c_min1[i,1]):
            t = 0
            alpha = np.append(alpha,np.arctan(t))
            continue
        if (BS_c[i,0] == BS_c_min1[i,0]):
            if ( BS_c[i,1] > BS_c_min1[i,1]):
                alpha = np.append(alpha,np.pi/2)
            else:
                alpha = np.append(alpha,-np.pi/2)
            continue  
        t = (BS_c[i,1]-BS_c_min1[i,1])/(BS_c[i,0]-BS_c_min1[i,0])
        alpha = np.append(alpha,np.arctan(t))

    v_id1 = np.array([])
    for i in range(0,alpha.size):
          v_id1 = np.append(v_id1,[v[i]*np.cos(alpha[i]),v[i]*np.sin(alpha[i])])
    v_id1 = v_id1.reshape(-1,2) 
    return v_id1

#### Covariance matrix P

We suppose that, at the beginning, the covariance matrix is a diagonal matrix like:
$$
P
=
\begin{bmatrix}0.01 & 0 & 0 & 0\\0 & 0.01 &  0 & 0\\0 & 0 & 0.01 & 0 \\ 0 & 0 &  0 & 0.01\end{bmatrix}
$$

#### Transition matrix A

Our next step is to design the state transition function. Recall that the state transition function is implemented as a matrix $A$ that we multiply with the previous state of our system to get the next state with the equation:

$x_k = Ax_{k-1} +Bu_k +w_{k-1}\quad\mathrm{}$

The state transition equations are:

$$
\begin{aligned}
x = 1x +  0y + \Delta t v_x + 0 v_y \\
y = 0x +  1y + 0v_x + \Delta t v_y \\
v_x = 0x + 0y + 1v_x + 0 v_y \\
v_y = 0x  + 0y + 0v_x + 1 v_y
\end{aligned}
$$
Laying it out that way shows us both the values and row-column organization required for $A$. We convert this to matrix-vector form:

$$
A
=
\begin{bmatrix}1 & 0 & \Delta t & 0\\0 & 1 &  0 & \Delta t\\0 & 0 & 1 & 0 \\ 0 & 0 &  0 & 1\end{bmatrix}
$$

#### Measurement matrix H
The measurement matrix $H$ defines how we go from the state variables $x$ to the measurements $y$ with the equation:

$y_k = Hx_{k} + v_{k} $

 In this case we have measurements for (x,y), so we will design $y_k$ as a vector of dimension 2x1. Our state variable $x$ is size 4x1. We can deduce the required size for $\textbf{H}$ by recalling that multiplying a matrix of size 2x1 by 1x4 yields a matrix of size 2x4. Thus,

$$
H
=
\begin{bmatrix}1 & 0 & 0 & 0\\0 & 1 &  0 & 0\\\end{bmatrix}
$$


We are now ready to implement the Kalman Filter:

In [5]:
#The method Kalman_Filter predicts the trajectory (that means the base cells C+1 that pick up the cars at time instant t+1) of one vehicle specified by its ID and calculate 
#the percentage of correct predictions


def Kalman_Filter(df, BS_loc, ID):
    df1 = df[df['ID'] == ID]

    #x and y coordinates of the BS location
    bs = BS_loc[['X', 'Y']].to_numpy()
    x_bs = BS_loc['X'].to_numpy()
    y_bs = BS_loc['Y'].to_numpy()

    #measurements is the vector with x and y coordinates of the vehicle location at each step
    measurements = df1[['car_x', 'car_y']].to_numpy() 

    #Speeds of the vehicle at each step
    v = df1['m/s'].to_numpy()

    #hexagonal cell associated with the x-y veichle location
    c_id1 = df1['C'].to_numpy()

    #hexagonal cell to which the vehicle will be connected in the next time step
    c1 = df1['C+1'].to_numpy()

    #hexagonal cell visited before the current one
    cmin1_id1 = df1['C-1'].to_numpy()

    v_id1 = velocity(c_id1,cmin1_id1,v,BS_loc)

    #time step of mobile movement 
    dt = 1 

    X = np.array([[measurements[0, 0]], [measurements[0, 1]],[v_id1[0,0]],[v_id1[0,1]]])
    P = np.diag((0.01, 0.01, 0.01, 0.01)) 
    A = np.array([[1, 0, dt , 0], [0, 1, 0, dt], [0, 0, 1, 0], [0, 0, 0, 1]]) 

    Q = np.eye(X.shape[0]) 
    B = np.eye(X.shape[0]) 
    U = np.zeros((X.shape[0],1)) 

    # Measurement matrices 
    Y = np.array([[X[0,0] + abs(np.random.randn(1)[0])], [X[1,0] + abs(np.random.randn(1)[0])]]) 
    H = np.array([[1, 0, 0, 0], [0, 1, 0, 0]]) 
    R = np.eye(Y.shape[0])

    # Number of iterations in Kalman Filter
    N_iter = len(df1)
    plt.figure(figsize=(15, 8))

    mes0 = []
    mes1 = []
    pred0 = []
    pred1 = []

    # Applying the Kalman Filter
    for i in np.arange(0, N_iter):
         
        X = np.array([[measurements[i, 0]], [measurements[i, 1]],[v_id1[i,0]],[v_id1[i,1]]])
        (X, P) = kf_predict(X, P, A, Q, B, U)
        pred0.append(X[0,0])#This is the predict value
        pred1.append(X[1,0])
    
        (X, P, K, IM, IS, LH) = kf_update(X, P, Y, H, R)

        Y = np.array([[X[0,0]],[X[1, 0]]])
        mes0.append(Y[0]) #This is the measured value
        mes1.append(Y[1])

    print("ID",ID)
    
    #Check if the base station that is nearest to the predicted coordinates is the real next base station (column C+1) 
    perc_real, index_BS_pred = corr_pred(BS_loc,N_iter,pred0,pred1,c1,bs)
    perc_pred, _ = corr_pred(BS_loc,N_iter,measurements[:, 0],measurements[:, 1],c1,bs)
    print("\n% of times the car is connected to the nearest BS (prediction) :",perc_real,"%")
    print("\n% of times the car is connected to the nearest BS (real mesurement) :",perc_pred,"%")
    return measurements,pred0, pred1, mes0, mes1,index_BS_pred

## Percentage of times the car is connected to the closest BS

In [6]:
 #Check if the base station that is nearest to the predicted coordinates is the real next base station (column C+1)
def corr_pred(BS_loc,N_iter,xcomp,ycomp,c1,bs):
  correct = 0
  error = 0
  index = -1
  index_BS_pred = []
  N = len(BS_loc)
  for i in np.arange(0, N_iter):
        pred = np.array([xcomp[i],ycomp[i]])
        min_dist = distance.euclidean(pred,bs[0])

        for j in np.arange(0,N-1):
            dist = distance.euclidean(pred,bs[j])
            if dist < min_dist:
                min_dist = dist
                index = j+1

        if ( c1[i] <= 0):
            continue
            
        if (index==c1[i]):
            correct = correct +1
        else: 
            error = error +1
        index_BS_pred.append(index)

  return (correct*100)/(correct+error), index_BS_pred